In [51]:
import os
import requests
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Connecting to Nebius LLM via API
Setup connection to Nebius API

In [52]:
#Find the key file

os.chdir("C:\\Users\\vital\\PythonStuff\\keys")
cwd = os.getcwd() 

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_8b_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
llama_70b_model ="meta-llama/Llama-3.3-70B-Instruct"
gemma_9b_model = "google/gemma-2-9b-it-fast"
Qwen2_5_72B_model = "Qwen/Qwen2.5-72B-Instruct"

In [53]:
userPrompt = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

# Function to chat with LLM model

In [54]:
def answer_with_llm(prompt: str,
                    system_prompt="You are an experience tutor with extensive experience in python code, C++, Java and software development lifecycle.",
                    max_tokens=512,
                    client=nebius_client,
                    model=llama_8b_model,
                    prettify=True,
                    stream=False,
                    displayMode="markdown",
                    temperature=None) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )
    if not stream:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return completion.choices[0].message.content

    # ✅ Streaming mode
    stream_resp = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True
    )

    
    print(f"🔁 Streaming from model: {model}\n")
    full_text = ""

    # Only create Markdown display handle if needed
    md_handle = None
    if displayMode == "markdown":
        md_handle = display(Markdown(""), display_id=True)

    # ✅ Single loop
    for chunk in stream_resp:
        if chunk.choices and chunk.choices[0].delta.content:
            token = chunk.choices[0].delta.content
            full_text += token

            if displayMode == "markdown":
                update_display(Markdown(full_text), display_id=md_handle.display_id)
            elif displayMode == "console":
                print(token, end="", flush=True)

    print("\n✅ Done streaming.\n")
    return None
    
    '''
    full_text = ""
    print(f"🔁 Streaming from model: {model}\n")
    for chunk in stream_resp:
        if chunk.choices and chunk.choices[0].delta.content:
            token = chunk.choices[0].delta.content
            print(token, end="", flush=True)
            full_text += token

    print("\n✅ Done streaming.\n")
    return full_text
    '''
    #if prettify:
    #    return prettify_string(completion.choices[0].message.content)
   # else:
    
    

In [55]:
def display_answer(prompt,model,stream,displayMode):
    summary = answer_with_llm(prompt, model=model, stream=stream, displayMode=displayMode)
    if not stream:
        if displayMode == "markdown":
            display(Markdown(summary))
        else:
            print(summary)
    #display(Markdown(summary))

In [56]:
display_answer(userPrompt,llama_8b_model,True,"markdown");

🔁 Streaming from model: meta-llama/Meta-Llama-3.1-8B-Instruct



This is a Python code snippet that uses a generator expression to yield a filtered list of book authors. Let's break it down:

- `yield from`: This is a special syntax in Python that allows a generator to yield values from another iterable. It's often used in conjunction with the `yield` keyword, which is used to create generators. 

- `{book.get("author") for book in books if book.get("author")}`: This is a set comprehension (or more accurately, a generator expression, since it's wrapped in parentheses and not curly brackets). 

  - `book.get("author")`: This gets the value associated with the key `"author"` from the `book` dictionary. If the key doesn't exist, it returns `None` instead of raising a `KeyError`.

  - `for book in books`: This iterates over each book in the `books` collection.

  - `if book.get("author")`: This is a conditional statement that filters out any `book` dictionaries that don't have an `"author"` key.

The entire expression is wrapped in a generator expression, which means it doesn't create a list of all the authors in memory at once. Instead, it lazily generates the authors one at a time as they're requested. 

Here's why this code might be used:

- **Efficiency**: If the `books` collection is very large, it would be inefficient to create a list of all the authors at once. This code allows you to iterate over the authors without having to store them all in memory.

- **Lazy Evaluation**: It only evaluates the expression when an author is requested, which can be useful if you're working with large datasets and only need to access a few authors at a time.

- **Readability**: The `yield from` syntax makes it clear that this code is yielding values from another iterable, which can make the code easier to read and understand.

Here's a simple example of how you might use this code:

```python
books = [
    {"title": "Book 1", "author": "Author 1"},
    {"title": "Book 2"},
    {"title": "Book 3", "author": "Author 3"}
]

for author in (book.get("author") for book in books if book.get("author")):
    print(author)
```

This will output:

```
Author 1
Author 3
```


✅ Done streaming.



In [57]:
display_answer(userPrompt,Qwen2_5_72B_model,True,"console")

🔁 Streaming from model: Qwen/Qwen2.5-72B-Instruct

Certainly! Let's break down the Python code snippet you provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set of unique author names from a list (or other iterable) of `books`.
   - `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary.
   - The `if book.get("author")` condition ensures that only books that have a non-`None` and non-empty author are included in the set. This is because `get` returns `None` if the key is not found, and `None` evaluates to `False` in a boolean context.

2. **`yield from`**:
   - `yield from` is a syntax in Python used to delegate to another generator or iterable. It allows you to yield all values from an iterable without having to write a loop.
   - In this case,

In [58]:
display_answer(userPrompt,gemma_9b_model,False,"markdown")

This code snippet demonstrates a powerful Python feature called **generators** and how they can be used in conjunction with **list comprehensions** to efficiently process data. Let's break it down step by step:

**Understanding the Components**

1. **`yield from`**: This is the heart of the generator. It tells Python that the function should yield one value at a time, pausing its execution until the next value is requested. The `from` keyword means it will yield values from another iterable (in this case, the set comprehension).

2. **`{book.get("author") for book in books if book.get("author")}`**:  This is a set comprehension, a concise way to create a set of values. It iterates through a collection named `books`.

    *  **`for book in books`**: This loops through each element (`book`) in the `books` collection.
    *  **`if book.get("author")`**:  This condition checks if the current `book` has a key called "author". The `.get("author")` method safely retrieves the value associated with the "author" key. If the key doesn't exist, it returns `None`. The `if` statement ensures that only books with a defined author are included.
    * **`book.get("author")`**:  For each book that passes the condition, it extracts and includes the value associated with the "author" key in the resulting set.

**Putting It Together**

This code snippet effectively does the following:

   1. **Filters:** It iterates through the `books` collection and keeps only those books that have an "author" specified.

   2. **Extracts:** For each filtered book, it extracts the author's name.

   3. **Yields:** It yields each extracted author's name one by one, creating a generator that produces a sequence of unique authors.

**Why Use This?**

* **Memory Efficiency:** Generators are memory-friendly because they don't store the entire result set in memory at once. They generate values on demand, which is particularly beneficial when dealing with large datasets.
* **Lazy Evaluation:** Values are generated only when requested. This can be advantageous if you're not sure if you'll need all the authors or if you want to process them gradually.

**Example Usage**

```python
books = [
    {"title":